In [1]:
import os
import json
import pprint 

os.chdir('../src/meshflow/')
os.getcwd()

'/home/kasra.keshavarz1/github-repos/meshflow/src/meshflow'

In [2]:
# necessary functions
def deep_merge(d1, d2):
    """
    Recursively merge d2 into d1.
    """
    for key, value in d2.items():
        if key in d1 and isinstance(d1[key], dict) and isinstance(value, dict):
            deep_merge(d1[key], value)  # Recursive merge for nested dictionaries
        else:
            d1[key] = value  # Overwrite or add new key-value pair
    return d1

In [3]:
info_block = {
    "vars": {
        "case_name": "MESH for Bow River at Calgary",
        "author": "Kasra Keshavarz",
        "location": "University of Calgary",
    },
    "comments": {
        "case_name": "01 TITLE",
        "author": "02 NAME",
        "location": "03 PLACE"
    },
    "formats": {}, # none means no change
    "columns": {
        "case_name": 3,
        "author": 3,
        "location": 3,
    }, # none means column 2
}

case_block = {
    "vars": {
        "case": {
            "centroid_lat": 45.2,
            "centroid_lon": -114.2,
            "reference_height_wndspd": 40.0,
            "reference_height_spechum_airtemp": 40.0,
            "reference_height_surface_roughness": 50.0,
            "ground_cover_flag": -1.0, # meaning land cover
            "ILW": 1, # MESH,CLASS>3.4 defaults to 1
            "NL": 30, # total number of subbasins
            "NM": 5, # total number of GRUs - 1
        },
    },
    "comments": {
        "case": '04 DEGLAT/DEGLON/ZRFM/ZRFH/ZBLD/GC/ILW/NL/NM'
    },
    "formats": {
        "case": {
            "centroid_lat": 2,
            "centroid_lon": 2,
            "reference_height_wndspd": 1,
            "reference_height_spechum_airtemp": 1,
            "reference_height_surface_roughness": 1,
            "ground_cover_flag": 1,
            "ILW": 0,
            "NL": 0,
            "NM": 0,
        },
    },
    "columns": {
        "case": {
            "centroid_lat": 4,
            "centroid_lon": 12,
            "reference_height_wndspd": 25,
            "reference_height_spechum_airtemp": 35,
            "reference_height_surface_roughness": 45,
            "ground_cover_flag": 52,
            "ILW": 60,
            "NL": 65,
            "NM": 70,
        },
    },
}

gru_block = {
    "vars": {
        "block1": {
            "veg": {
                "class": "needleleaf",
            },
        },
        "block2": {
            "veg": {
                "class": "broadleaf",
            },
        },
        "block3": {
            "veg": {
                "class": "crops",
            },
        },
        "block4": {
            "veg": {
                "class": "grass",
            },
        },
        "block5": {
            "veg": {
                "class": "barrenland",
            },
        },
    },
}

In [4]:
# load the default values for each GRU
with open('./templates/default_parameters.json', 'r') as file:
    data = json.load(file)

# 
for block in gru_block['vars'].keys():

    # take a copy of the default values
    dummy = data.copy()
    
    # update the block dictionary
    gru_block['vars'][block] = deep_merge(dummy, gru_block['vars'][block])

In [ ]:
from jinja2 import Environment, FileSystemLoader

def raise_helper(msg):
    raise Exception(msg)

environment = Environment(
    loader=FileSystemLoader("templates/"),
    trim_blocks=True,
    lstrip_blocks=True,
    line_comment_prefix='##',
)
environment.globals['raise'] = raise_helper

template = environment.get_template("MESH_parameters_CLASS.ini.jinja")

# create content
content = template.render(
    info_block=info_block,
    case_block=case_block,
    gru_block=gru_block,
    variables="vars",
    comments="comments",
    formats="formats",
    columns="columns",
)

print(content)